# Plain Text Tables with Real Data
> Sample Covid-19 API Data pulls into pain text tables

- toc: true
- badges: true
- comments: true
- hide: true
- categories: [general, tables, data, covid19]

In [1]:
#hide

# PyPI
from bs4 import BeautifulSoup as BS
import requests

# Standard
import pprint
import json
import re

# SUPPORTING FUNCTIONS
def build_title(title: str) -> str:
    return f'-------------------------------\n{title}\n-------------------------------\n'

In [2]:
#hide

import requests, json

query_url = 'https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetResurgenceData?regionID=8&daysIncluded=3' #+ selectedRegion + '&daysIncluded=' + chartRange

page = requests.get(query_url)

print(build_title('CLI ADMISSIONS'))
pprint.pprint(page.json()['CLIAdmissions'])

print(build_title('COUNTRY TEST POSITIVITY REPORTS'))
pprint.pprint(page.json()['CountyTestPositivityReports'])

print(build_title('HOSPITAL AVAILABILITY'))
pprint.pprint(page.json()['HospitalAvailability'])

print(build_title('HOSPITAL BEDS IN USE AVG'))
pprint.pprint(page.json()['HospitalBedsInUseAvg'])

print(build_title('TEST POSITIVITY'))
pprint.pprint(page.json()['TestPositivity'])

print(build_title('LAST UPDATED DATE'))
pprint.pprint(page.json()['lastUpdatedDate'])

print(build_title('REGION METRICS'))
pprint.pprint(page.json()['regionMetrics'])

-------------------------------
CLI ADMISSIONS
-------------------------------

[{'CLIAdmissionsRA': 47.0,
  'regionDescription': 'West Suburban',
  'regionID': 8,
  'reportDate': '2020-11-26T00:00:00'},
 {'CLIAdmissionsRA': 47.0,
  'regionDescription': 'West Suburban',
  'regionID': 8,
  'reportDate': '2020-11-27T00:00:00'},
 {'CLIAdmissionsRA': 48.0,
  'regionDescription': 'West Suburban',
  'regionID': 8,
  'reportDate': '2020-11-28T00:00:00'},
 {'CLIAdmissionsRA': 49.0,
  'regionDescription': 'West Suburban',
  'regionID': 8,
  'reportDate': '2020-11-29T00:00:00'}]
-------------------------------
COUNTRY TEST POSITIVITY REPORTS
-------------------------------

[{'countyTestPositivities': [{'CountyName': 'DuPage',
                              'dailyPositivity': 0.0,
                              'positive_test': 660,
                              'positivityRollingAvg': 11.8,
                              'regionID': 8,
                              'totalTest': 5301},
            

In [3]:
print(build_title('COUNTY TEST POSITIVITY REPORTS'))

query_url = 'https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetResurgenceData?regionID=8&daysIncluded=5' #+ selectedRegion + '&daysIncluded=' + chartRange
page = requests.get(query_url)

positivity_rates = page.json()['CountyTestPositivityReports']
print('SINGLE SAMPLE')
pprint.pprint(positivity_rates[0])
print()

print('+-----------------------------------------------------------+')
print('| DATE       |  NAME      | POS    | TOTAL  | AVG   | 7 DAY |')
print('|-----------------------------------------------------------|')
for counties in positivity_rates:
    for i in counties['countyTestPositivities']:
        date = counties['reportDate'][:10]
        name = i['CountyName']
        name_fill = ' '*(10 - len(name))
        positive = i['positive_test']
        positive_fill = ' '*(6 - len(str(positive)))
        total = i['totalTest']
        total_fill = ' '*(6 - len(str(total)))
        daily_avg = round(float(i['positive_test']/i['totalTest']*100), 1) # positive tests / total tests * 100, then rounded to 1 decimal place
        daily_avg_fill = ' ' *(5 - len(str(daily_avg)))
        seven_day_avg = i['positivityRollingAvg']
        seven_fill = ' '*(5 - len(str(seven_day_avg)))
        print(f'| {date} | {name}{name_fill} | {positive}{positive_fill} | {total}{total_fill} | {daily_avg}{daily_avg_fill} | {seven_day_avg}{seven_fill} |')
print('+-----------------------------------------------------------+')

-------------------------------
COUNTY TEST POSITIVITY REPORTS
-------------------------------

SINGLE SAMPLE
{'countyTestPositivities': [{'CountyName': 'DuPage',
                             'dailyPositivity': 0.0,
                             'positive_test': 777,
                             'positivityRollingAvg': 12.3,
                             'regionID': 8,
                             'totalTest': 6411},
                            {'CountyName': 'Kane',
                             'dailyPositivity': 0.0,
                             'positive_test': 517,
                             'positivityRollingAvg': 16.3,
                             'regionID': 8,
                             'totalTest': 3790}],
 'reportDate': '2020-11-24T00:00:00'}

+-----------------------------------------------------------+
| DATE       |  NAME      | POS    | TOTAL  | AVG   | 7 DAY |
|-----------------------------------------------------------|
| 2020-11-24 | DuPage     | 777    | 6411   | 12

In [4]:
import datetime
from datetime import date

# TABLE PARAMETERS
line_length = 59

print(build_title('COUNTY TEST POSITIVITY REPORTS'))

query_url = 'https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetResurgenceData?regionID=8&daysIncluded=10' #+ selectedRegion + '&daysIncluded=' + chartRange
page = requests.get(query_url)

positivity_rates = page.json()['CountyTestPositivityReports']
print('SINGLE SAMPLE')
pprint.pprint(positivity_rates[-1])
behind = datetime.datetime.strptime(positivity_rates[-1]['reportDate'], "%Y-%m-%dT%H:%M:%S")
behind_str = f'CURRENT DATE: {date.today()} (Report is {(date.today() - behind.date()).days} days *behind*)'
behind_fill = ' '*(57 - len(behind_str))
print()
print(f'+{"="*line_length}+')
print(f'| {behind_str}{behind_fill} |')
print(f'|{"="*line_length}|')
print('| NAME       | DATE       | POS    | TOTAL  | AVG   | 7 DAY |')
print(f'|{"-"*line_length}|')
number_of_counties = len(positivity_rates[0]['countyTestPositivities'])
for n in range(number_of_counties):
    print_name = True
    for counties in positivity_rates:
        county = counties['countyTestPositivities'][n]#[n-1]
        date = counties['reportDate'][:10]
        name = county['CountyName'] if print_name == True else ''
        name_fill = ' '*(10 - len(name))
        positive = county['positive_test']
        positive_fill = ' '*(6 - len(str(positive)))
        total = county['totalTest']
        total_fill = ' '*(6 - len(str(total)))
        daily_avg = round(float(county['positive_test']/county['totalTest']*100), 1) # positive tests / total tests * 100, then rounded to 1 decimal place
        daily_avg_fill = ' ' *(5 - len(str(daily_avg)))
        seven_day_avg = county['positivityRollingAvg']
        seven_fill = ' '*(5 - len(str(seven_day_avg)))
        print(f'| {name}{name_fill} | {date} | {positive}{positive_fill} | {total}{total_fill} | {daily_avg}{daily_avg_fill} | {seven_day_avg}{seven_fill} |')
        print_name = False
    print(f'+{"-"*line_length}+')

-------------------------------
COUNTY TEST POSITIVITY REPORTS
-------------------------------

SINGLE SAMPLE
{'countyTestPositivities': [{'CountyName': 'DuPage',
                             'dailyPositivity': 0.0,
                             'positive_test': 431,
                             'positivityRollingAvg': 11.4,
                             'regionID': 8,
                             'totalTest': 4188},
                            {'CountyName': 'Kane',
                             'dailyPositivity': 0.0,
                             'positive_test': 265,
                             'positivityRollingAvg': 15.0,
                             'regionID': 8,
                             'totalTest': 1750}],
 'reportDate': '2020-11-29T00:00:00'}

+===========================================================+
| CURRENT DATE: 2020-12-02 (Report is 3 days *behind*)      |
|===========================================================|
| NAME       | DATE       | POS    | TOTAL  | AV